In [ ]:
import re # Importa el módulo para expresiones regulares
from datetime import datetime, timedelta # Importa clases para manejar fechas y tiempos
from typing import Dict, List, Any, Optional
from logging import warning

import requests
import pandas as pd
import re
from openai import OpenAI
import json
import os
import unicodedata

import time


In [ ]:
api_key = "sk-proj-bUC23dKTYqlLzAufh...bF-B_iWuXD.."
client = OpenAI(api_key=api_key)
LLM_CACHE = {}

In [ ]:
import openai
import json
from typing import Dict, Any, Optional

# Asegúrate de haber configurado tu clave de API de OpenAI.
# Es una buena práctica usar variables de entorno para esto.
# import os
# openai.api_key = os.getenv("OPENAI_API_KEY")

# Si no tienes variables de entorno, puedes descomentar la siguiente línea
# y colocar tu clave directamente, pero con precaución.


def parser_openai(query: str) -> Optional[Dict[str, Any]]:
    """
    Analiza una consulta de lenguaje natural para subvenciones usando la API de OpenAI.

    Args:
        query (str): La consulta de búsqueda del usuario en lenguaje natural.

    Returns:
        Optional[Dict[str, Any]]: Un diccionario con los parámetros de búsqueda extraídos,
                                  o None si la llamada a la API falla.
    """
    system_prompt = f'''Eres un asistente experto en la clasificación de subvenciones. Tu tarea es analizar una consulta de un usuario y extraer parámetros clave para refinar una búsqueda de subvenciones.
                A continuación, se te proporcionan varios diccionarios de mapeo. Cada diccionario contiene descripciones y palabras clave asociadas a un ID numérico o un código.
                Debes leer la QUERY del usuario, identificar qué clasificaciones son relevantes para cada PARAMETRO y devolver el ID o código correspondiente.
                ##

                # Mapeo de nombres de regiones/provincias de España a sus IDs numéricos.
                # Las claves son strings en minúsculas y los valores son listas que contienen el ID.
                regiones_map = {{
                "a coruña": [4], "lugo": [5], "ourense": [6], "pontevedra": [7],
                "galicia": [3], "asturias": [9], "principado de asturias": [8],
                "cantabria": [11], "cantabria": [10],
                "noroeste": [2], "araba/álava": [14], "gipuzkoa": [15],
                "bizkaia": [16], "pais vasco": [13], "navarra": [18],
                "comunidad foral de navarra": [17], "la rioja": [20],
                "la rioja": [19],
                "huesca": [22], "teruel": [23], "zaragoza": [24],
                "aragon": [21], "noreste": [12], "madrid": [27],
                "comunidad de madrid": [26], "centro (es)": [28],
                "ávila": [30], "burgos": [31], "león": [32],
                "palencia": [33], "salamanca": [34], "segovia": [35],
                "soria": [36], "valladolid": [37], "zamora": [38],
                "castilla y leon": [29], "albacete": [40], "ciudad real": [41],
                "cuenca": [42], "guadalajara": [43], "toledo": [44],
                "castilla la mancha": [39], "badajoz": [46], "cáceres": [47],
                "extremadura": [45], "barcelona": [50], "girona": [51],
                "lleida": [52], "tarragona": [53], "cataluña": [49],
                "alicante": [55], "castellón": [56], "valencia": [57],
                "comunidad valenciana": [54], "eivissa y formentera": [59],
                "mallorca": [60], "menorca": [61], "illes balears": [58],
                "este": [48], "almería": [64], "cádiz": [65], "córdoba": [66],
                "granada": [67], "huelva": [68], "jaén": [69], "málaga": [70],
                "sevilla": [71], "andalucia": [63], "murcia": [73],
                "region de murcia": [72], "ceuta": [75], "ciudad autonoma de ceuta": [74],
                "melilla": [77], "ciudad autonoma de melilla": [76],
                "sur": [62], "el hierro": [80], "fuerteventura": [81],
                "gran canaria": [82], "la gomera": [83], "la palma": [84],
                "lanzarote": [85], "tenerife": [86], "canarias": [79],
                "españa": [1], "extra-regio nuts 1": [87]
                }}


                # Mapeo de palabras clave de tipos de beneficiario a sus IDs numéricos.
                # Los IDs están en listas, lo que permite flexibilidad para futuras expansiones
                tipos_beneficiario_comunes_map = {{
                    "Esta categoría incluye a personas físicas que no desarrollan ninguna actividad económica. Engloba a particulares, ciudadanos en general, individuos, estudiantes, jubilados, desempleados, pensionados, familias y hogares que buscan ayudas o beneficios.": [1],
                    "Esta categoría se refiere a personas jurídicas que no persiguen un fin de lucro o no realizan una actividad económica lucrativa. Incluye a asociaciones, fundaciones, organizaciones sin ánimo de lucro (ONGs), clubes deportivos, federaciones, confederaciones, partidos políticos y colegios profesionales.": [2],
                    "Engloba a Pequeñas y Medianas Empresas (PYMES) y a personas físicas (autónomos) que sí desarrollan una actividad económica. Aquí se incluyen autónomos, profesionales independientes, emprendedores, pequeños negocios, microempresas, sociedades limitadas (SL) y start-ups.": [3],
                    "Esta categoría está dirigida a grandes empresas, corporaciones, multinacionales y otras entidades de gran envergadura o tamaño económico. Son organizaciones con una gran capacidad productiva y un número elevado de empleados.": [4]
                }}



                # Mapeo de palabras clave de instrumentos de ayuda a sus IDs numéricos.
                # Los IDs están en listas, permitiendo asociar múltiples palabras a un mismo ID.
                instrumentos_map = {{
                    "SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPRESTACIÓN. Ayuda económica no reembolsable, también conocida como subvención directa, que se otorga sin esperar devolución ni contraprestación. Se vincula a un objetivo concreto como la innovación, el empleo, o el desarrollo regional. Palabras clave: subvención, ayuda directa, entrega dineraria, no reembolsable, sin contraprestación.": 1,
                    "PRÉSTAMO. Instrumento financiero basado en la entrega de capital reembolsable, generalmente sujeto a intereses y plazos de amortización. Puede tener tipo de interés fijo o variable. Frecuente en programas de financiación para inversiones, digitalización o crecimiento empresarial. Palabras clave: préstamo, devolución, intereses, amortización, financiación": 2,
                    "GARANTÍA. Instrumento de aval, fianza o respaldo financiero proporcionado por una entidad para asegurar el cumplimiento de obligaciones o facilitar el acceso a otras fuentes de financiación. Se usa mucho en licitaciones, proyectos de inversión o emprendimiento. Palabras clave: garantía, aval, respaldo, fianza, seguridad financiera.": 4,
                    "VENTAJA FISCAL. Conjunto de incentivos en materia de impuestos o tributos, como deducciones fiscales, bonificaciones, exenciones o aplazamientos. Permiten a empresas o autónomos reducir su carga fiscal. Muy frecuente en políticas de I+D+i o empleo. Palabras clave: ventaja fiscal, deducción, bonificación, exención, ahorro tributario.": 5,
                    "APORTACIÓN DE FINANCIACIÓN DE RIESGO. Modalidad de inversión que implica asumir riesgo empresarial, como el capital riesgo o el venture capital. El apoyo se hace mediante la entrada al capital social o financiación subordinada. Muy usada en startups o proyectos con alto potencial de crecimiento. Palabras clave: capital riesgo, inversión, venture capital, participación, financiación de riesgo.": 6,
                    "OTROS INSTRUMENTOS DE AYUDA. Categoría abierta para apoyos no monetarios directos como consultoría gratuita, asesoramiento técnico, acceso a espacios físicos o servicios sin coste. Incluye cualquier forma de ayuda que no sea préstamo, subvención, garantía, fiscalidad o capital. Palabras clave: asesoramiento, consultoría, cesión, soporte no financiero, apoyo institucional.": 7
                }}

                # Mapeo de palabras clave de finalidades de política de gasto a sus IDs numéricos.
                finalidades_map = {{
                    "ACCESO A LA VIVIENDA Y FOMENTO DE LA EDIFICACIÓN. Ayudas y programas dirigidos a facilitar la adquisición, el alquiler o la rehabilitación de viviendas, así como al impulso y financiación de proyectos de construcción de nuevas edificaciones. Palabras clave: vivienda, alquiler, hipoteca, rehabilitación, edificación.": 8,
                    "AGRICULTURA, PESCA Y ALIMENTACIÓN. Programas de apoyo a la agricultura, la ganadería, la silvicultura, la pesca y todas las actividades relacionadas con la producción, transformación y comercialización de alimentos. Palabras clave: agroalimentario, ganadería, agricultura, pesca, alimentos.": 12,
                    "COMERCIO, TURISMO. Iniciativas y fondos destinados a impulsar el comercio minorista y mayorista, la promoción turística de destinos y servicios, y el apoyo específico a Empresas en su desarrollo y crecimiento del sector turismo. Palabras clave: comercio, turismo, hostelería, marketing comercial.": 14,
                    "COOPERACIÓN INTERNACIONAL PARA EL DESARROLLO Y CULTURAL. Proyectos y fondos orientados a la colaboración con otros países para su desarrollo económico y social, así como al fomento y difusión de la cultura a nivel internacional. Palabras clave: cooperación internacional, ayuda exterior, desarrollo global, cultura exterior, relaciones internacionales.": 20,
                    "CULTURA. Ayudas y programas para la promoción, conservación y difusión del patrimonio cultural, las artes escénicas, las bellas artes, el cine, la música, la literatura y otras manifestaciones culturales. Palabras clave: patrimonio, arte, cine, música, literatura.": 11,
                    "DEFENSA. Programas y presupuestos destinados a la seguridad y defensa nacional, incluyendo el equipamiento militar, la formación de personal y las operaciones de seguridad y protección. Palabras clave: defensa, militar, ejército, armamento, estrategia nacional.": 2,
                    "DESEMPLEO. Ayudas y prestaciones dirigidas a personas en situación de desempleo, incluyendo subsidios, prestaciones por desempleo y programas de reinserción laboral. Palabras clave: paro, subsidio, prestación por desempleo, reinserción, desempleado.": 7,
                    "EDUCACION. Fondos y programas dedicados a la financiación de centros educativos, becas para estudiantes, formación profesional, educación superior y todas las actividades relacionadas con la enseñanza y el aprendizaje. Palabras clave: educación, beca, escolar, universidad, formación académica.": 10,
                    "FOMENTO DEL EMPLEO. Iniciativas y ayudas para la creación de empleo, el fomento del autoempleo, la formación y cualificación profesional, y el apoyo a la contratación de colectivos específicos. Palabras clave: contratación, autoempleo, inserción laboral, empleabilidad, creación de empleo.": 6,
                    "INDUSTRIA Y ENERGÍA. Ayudas y programas para el desarrollo de la industria, la innovación tecnológica en el sector industrial, la eficiencia energética y el fomento de fuentes de energía sostenibles. Palabras clave: industria, energía, eficiencia energética, fábrica, energía renovable.": 13,
                    "INFORMACIÓN NO DISPONIBLE. Esta categoría se utiliza cuando no se dispone de información específica o suficiente para clasificar la ayuda en ninguna de las otras áreas definidas. Palabras clave: desconocido, sin categorizar, información ausente, no disponible.": 21,
                    "INFRAESTRUCTURAS. Inversiones y proyectos para el desarrollo y mantenimiento de infraestructuras de transporte (carreteras, ferrocarriles, puertos, aeropuertos), energéticas, hidráulicas o de telecomunicaciones. Palabras clave: carreteras, infraestructuras, transporte, puertos, telecomunicaciones.": 16,
                    "INVESTIGACIÓN, DESARROLLO E INNOVACIÓN. Apoyo a proyectos de investigación científica, desarrollo tecnológico y actividades de innovación en todos los sectores, buscando el avance del conocimiento y la aplicación de nuevas tecnologías. Palabras clave: I+D, innovación, ciencia, desarrollo tecnológico, investigación aplicada.": 17,
                    "JUSTICIA. Programas y fondos relacionados con la administración de justicia, los sistemas judiciales, la asistencia legal y los servicios penitenciarios. Palabras clave: justicia, tribunales, legal, juzgado, penitenciario.": 1,
                    "OTRAS ACTUACIONES DE CARÁCTER ECONÓMICO. Categoría amplia que engloba subvenciones y ayudas no clasificables en otros sectores específicos, pero que tienen un claro impacto o finalidad económica, como apoyo a empresas en general, desarrollo regional, etc. Palabras clave: subvenciones, incentivo empresarial, desarrollo economico, crecimiento.": 18,
                    "OTRAS PRESTACIONES ECONÓMICAS. Incluye diversas ayudas económicas que no se ajustan a las categorías de empleo, vivienda o dependencia, como ayudas a familias, a la natalidad, o prestaciones económicas por situaciones especiales. Palabras clave: natalidad, familia numerosa, prestación especial, ayuda puntual, situación excepcional.": 4,
                    "SANIDAD. Programas y fondos destinados a la atención sanitaria, la prevención de enfermedades, la salud pública, la investigación médica y la mejora de los servicios de salud. Palabras clave: salud, sanidad, atención médica, prevención, sistema sanitario.": 9,
                    "SEGURIDAD CIUDADANA E INSTITUCIONES PENITENCIARIAS. Ayudas y presupuestos para la seguridad pública, las fuerzas y cuerpos de seguridad, la prevención del delito, y la gestión y funcionamiento de las instituciones penitenciarias. Palabras clave: seguridad, policía, vigilancia, delincuencia, prisión.": 3,
                    "SERVICIOS SOCIALES Y PROMOCIÓN SOCIAL. Programas y prestaciones dirigidos a colectivos vulnerables, promoción de la inclusión social, atención a la dependencia, servicios para personas mayores o con discapacidad, y otras iniciativas de bienestar social. Palabras clave: servicios sociales, dependencia, inclusión, personas mayores, discapacidad.": 5,
                    "SIN INFORMACION ESPECIFICA. Similar a 'INFORMACIÓN NO DISPONIBLE', esta categoría se usa cuando la temática de la subvención no está claramente definida o especificada dentro de las categorías preestablecidas. Palabras clave: sin información, sin especificar, indeterminado, categoría desconocida.": 19,
                    "SUBVENCIONES AL TRANSPORTE. Ayudas específicas destinadas a fomentar el uso del transporte público, la mejora de infraestructuras de transporte o la subvención de billetes o abonos para usuarios. Palabras clave: transporte público, billete subvencionado, movilidad urbana, bono transporte, accesibilidad vial.": 15
                }}

                ## INSTRUCCIONES PARA LA SALIDA:

                1.  Analiza la consulta de usuario.
                2.  Para cada PARAMETRO (regiones, tiposBeneficiario, instrumentos, finalidad), determina la mejor coincidencia. IMPORTANTE: tiposBeneficiario y regiones pueden tener dos opciones como máximo. Las que mejor clasifiquen la query.
                2.1. Para clasificar la finalidad, identifica primero el SECTOR o ACTIVIDAD PRINCIPAL. Si la consulta menciona una empresa, producto o servicio concreto (ej. librería, granja, clínica, editorial), busca en qué sector de la economía encaja (Cultura, Agricultura, Sanidad, Industria, etc.) antes de considerar el tamaño o el tipo de beneficiario.
                2.2. Para clasificar instrumentos, prioriza las palabras clave y expresiones exactas de la consulta. Si se mencionan términos como “subvención”, “ayuda directa”, “aportación no reembolsable” asignar a SUBVENCIÓN. Si se mencionan “préstamo”, “crédito”, “leasing”, “renting”, “financiación de activos” asignar a PRÉSTAMO. Si se mencionan “garantía”, “aval”, “fianza” asignar a GARANTÍA. Si se mencionan deducciones, bonificaciones o exenciones fiscales asignar a VENTAJA FISCAL. Si se habla de capital riesgo, inversión o venture capital asignar a APORTACIÓN DE FINANCIACIÓN DE RIESGO. Si no se identifica ninguna palabra clave de las categorías anteriores, asignar OTROS INSTRUMENTOS DE AYUDA. No asumir que “financiación” significa automáticamente subvención; deducir según contexto y sector.
                3.  Si una consulta sugiere múltiples opciones para un parámetro (por ejemplo, "Castilla y León y Madrid"), incluye todos los IDs relevantes en una lista. Igual para tipos_beneficiario. maximo dos.
                4.  Si un parámetro (regiones, tiposBeneficiario, instrumentos, finalidad) no se menciona en la consulta y no se puede inferir, no lo agregues al diccionario de salida. finalidad siempre debe tener un valor, no puede ser NULL.
                5.  También necesito que extraigas una cadena con maximo 10 palabras clave de la query, ignorando stopwords. Usa la clave "descripcion".
                6.  Si el QUERY contiene un entero de 6 digitos empezando con el numero 8, lo mas probable es que se trate del numero de convocatoria. así que colócalo como el parámetro "numeroConvocatoria". Al mismo tiempo deja los demas parametros(regiones, tiposBeneficiario, instrumentos, finalidad) en None.
                7.  El formato de salida debe ser un único objeto JSON. NO AGREGUES MÁS INFORMACIÓN. SOLO EL DICCIONARIO.
                8.  Hay algunos parámetros que son fijos y no cambian, pero quiero que los incluyas siempre:
                    - "descripcionTipoBusqueda": 2
                    - "order": "fechaRecepcion"
                    - "direccion": "desc"
                    - "vpd": "GE"
                    - "page": 0
                    - "pageSize": 25

                EJEMPLO de formato de salida:

                {{
                  "descripcion": "beca doctorado estudiante",
                  "numeroConvocatoria": 812345,
                  "regiones": [27],
                  "finalidad": 8,
                  "instrumentos": 1,
                  "tiposBeneficiario": [1, 3]
                  "descripcionTipoBusqueda": 2,
                  "order": "fechaRecepcion",
                  "direccion": "desc",
                  "vpd": "GE",
                  "page": 0,
                  "pageSize": 25
                }}


                '''

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"QUERY: {query}"}
            ],
            temperature=0.0
        )

        # Extrae el contenido de la respuesta y limpia el formato JSON si es necesario
        content = response.choices[0].message.content

        # El modelo a veces añade ```json ... ```, hay que eliminarlo
        if content.startswith("```json") and content.endswith("```"):
            json_string = content.strip("```json").strip("```").strip()
        else:
            json_string = content.strip()

        # Cargar la cadena JSON en un diccionario de Python
        return json.loads(json_string)

    except openai.APIError as e:
        print(f"Error de la API de OpenAI: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error al decodificar la respuesta JSON: {e}")
        print(f"Respuesta recibida: {content}")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return None



In [114]:
'''Aqui hay algunas funciones auxiliares basadas en el trabajo de Carmen para comunicarnos con el sitio web de subvenciones.

'''

def buscar_convocatorias(**params): #esta funcion toma el diccionario de palabras obtenido de la función "parsear_busqueda_subvenciones" y hace un request con la API al sitio de subvenciones.
#se obtiene un DF con las convocatorias que satisfacen los argumentos del diccionario.
    base_url = "https://www.pap.hacienda.gob.es/bdnstrans/api/convocatorias/busqueda"
    headers = {"Accept": "application/json"}
    page_size = params.get("pageSize", 25)
    max_paginas = 3
    resultados = []
    params.setdefault("vpd", "GE")
    params.setdefault("pageSize", page_size)
    for pagina in range(0, max_paginas):
        params["page"] = pagina
        try:
            response = requests.get(base_url, params=params, headers=headers)
            response.raise_for_status()
            if "application/json" in response.headers.get("Content-Type", ""):
                data = response.json()
                convocatorias = data.get("convocatorias", data.get("content", []))
                if not convocatorias:
                    break
                resultados.extend(convocatorias)
                time.sleep(0.5)
        except Exception as e:
            break
    return pd.DataFrame(resultados)

def obtener_convocatoria_por_id(num_conv): #esta funcion toma un numero especifico de convocatoria y realiza un request a la API, la cual devuelve un diccionario con información específica de la convocatoria.
    base_url = "https://www.infosubvenciones.es/bdnstrans/api/convocatorias"
    params = {"vpd": "GE", "numConv": str(num_conv)}
    headers = {"Accept": "application/json"}
    r = requests.get(base_url, params=params, headers=headers)
    if r.status_code == 200 and "application/json" in r.headers.get("Content-Type", ""):
        return r.json()
    else:
        return None

def descargar_documento_pdf(documento_id, nombre): #esta funcion requiere el documento_id y nombre y descarga en ruta relativa el pdf en cuestión.
    url = f"https://www.infosubvenciones.es/bdnstrans/api/convocatorias/documentos?idDocumento={documento_id}"
    respuesta = requests.get(url)
    if respuesta.status_code == 200:
        ruta_relativa = os.path.join("documentos", nombre)
        ruta_absoluta = os.path.abspath(ruta_relativa)
        os.makedirs(os.path.dirname(ruta_absoluta), exist_ok=True)
        with open(ruta_absoluta, "wb") as f:
            f.write(respuesta.content)
        return ruta_absoluta
    return None


def extraer_datos_convocatoria(numero, client):
    dic = obtener_convocatoria_por_id(numero)  # tu función que devuelve el dict
    presupuesto = dic.get('presupuestoTotal')

    if isinstance(presupuesto, (int, float)):
        presupuesto = f"{presupuesto:,.2f}"



    inicio = dic.get('fechaInicioSolicitud') or dic.get('textInicio')
    final = dic.get('fechaFinSolicitud') or dic.get('textFin')

    ''' # Si no hay fechas estructuradas, intentar inferir con LLM
    if not inicio and not final:
        inicio, final = inferir_plazo_via_llm(dic, client)'''

    return pd.Series([presupuesto, inicio, final],
                     index=['presupuesto_total', 'inicio', 'final'])





def data_frame_resumen(query):
    try:
        filtros = parser_openai(query)
        # Limpia campos con valor None
        filtros2 = {k: v for k, v in filtros.items() if v is not None}
        print(filtros2)
        df = buscar_convocatorias(**filtros2)

        if df.empty:
            print("No se encontraron convocatorias.")
            return pd.DataFrame()

        df.drop(columns=['mrr', 'descripcionLeng', 'nivel3', 'codigoInvente', 'id'], inplace=True)

        df['fechaRecepcion'] = pd.to_datetime(df['fechaRecepcion'], errors='coerce')
        fecha_limite = pd.Timestamp.now() - pd.DateOffset(years=1)
        df = df[df['fechaRecepcion'] >= fecha_limite]

        if 'numeroConvocatoria' not in df.columns or df['numeroConvocatoria'].isnull().all():
            print("No hay datos válidos de número de convocatoria.")
            return pd.DataFrame()

        df[['presupuesto_total', 'inicio', 'final']] = df['numeroConvocatoria'].apply(
            lambda x: pd.Series(extraer_datos_convocatoria(x,client))
        )
        df['estado'] = df['numeroConvocatoria'].apply(obtener_edo_LLM, args=(client,))




        return df

    except Exception as e:
        print(f"Ocurrió un error procesando los datos: {e}")
        return pd.DataFrame()




def obtener_edo_LLM(convocatoria_id, client):
    """
    Determina si la convocatoria está abierta, cerrada o desconocida usando LLM.
    """
    dic = obtener_convocatoria_por_id(convocatoria_id)
    tipo= dic.get('tipoConvocatoria', '').lower()
    today = datetime.now().date()
    fecha_recepcion = dic.get("fechaRecepcion")
    fecha_inicio = dic.get("fechaInicioSolicitud")
    fecha_fin = dic.get("fechaFinSolicitud")

    if 'concesión directa' in tipo or 'nominativa' in tipo:
       return 'cerrada-no-publica'

    # Si ya tenemos fecha de fin estructurada, podemos devolver rápido sin LLM
    if fecha_fin:
        try:
            fecha_fin_dt = datetime.strptime(fecha_fin, "%Y-%m-%d").date()
            if today > fecha_fin_dt:
                return "cerrada"
            else:
                return "abierta"
        except:
            pass

      # Prepara textos relevantes con contexto
    textos = []

    # textFin
    if dic.get("textFin"):
        textos.append({
            "origen": "textFin",
            "texto": dic["textFin"],

        })

    # anuncios
    for a in dic.get("anuncios", []):


        if a.get("titulo"):
            textos.append({
                "origen": "ANUNCIO",
                "texto": a["texto"],
            })

    # documentos
    for d in dic.get("documentos", []):
        if d.get("descripcion"):
            textos.append({
                "origen": "DOCUMENTO",
                "texto": d["descripcion"],

            })





    system_msg = """
    Eres un asistente que analiza la información de una convocatoria de subvenciones
    y determina su estado actual.

    Devuelve SOLO un JSON válido:
    {"estado": "abierta" | "cerrada_" | "desconocido"}

    Reglas en estricto orden:

    Se te da un texto con varios origenes: textFin,anuncios,documentos.


    1) Revisa el TEXTO de textFin: si contiene una FECHA EXPLÍCITA de cierre (DD/MM/AAAA, DD-MM-AAAA, AAAA-MM-DD,
      o “DD de <mes> de AAAA”), usa esa fecha para comparar con HOY. Si HOY > fecha → "cerrada",
      en otro caso → "abierta".


    2) Si el TEXTO de textFin no tiene una fecha explícita: busca en el TEXTO de DOCUMENTO.
   - Si aparece cualquier frase como “cierre de convocatoria”, “plazo finalizado”, “convocatoria cerrada”,
     “resolución de cierre” o “plazo concluido” → "cerrada".
   - Si no hay nada de esto → continuar al siguiente paso.

   3) Si en el TEXTO de DOCUMENTO no hay nada de cierre: busca en el TEXTO de ANUNCIO.

   - Si aparece una regla relativa como “[numero] días naturales/hábiles desde apertura” o “vigencia de [numero] días”:
        a) Utiliza la FECHA DE RECEPCION que se te proporciona antes de TEXTOS RELEVANTES como apertura.
        b) Sumar el número de días indicados
        c) Comparar con HOY:
           - Si HOY > fecha calculada → "cerrada".
           - En otro caso → "abierta".
   - Si no hay nada de esto → continuar al siguiente paso.


    4) Si el TEXTO de textFin dice algo como “hasta agotar fondos/crédito”→ "abierta".
    -si no hay nada de esto-> continuar al siguiente paso.

    5) Si el TEXTO de textFin contiene algo como “[número en letra o dígito] días naturales/hábiles desde la publicación de [evento]”:

      a) utiliza FECHA DE RECEPCION que se te proporciona antes de textos relevantes.

      b) Convierte el número de días a entero.

      c) Suma los días a la FECHA DE RECEPCION y obtén la fecha de cierre.

      d) Si HOY > fecha de cierre → "cerrada", en otro caso → "abierta".


    No expliques. No añadas campos. SOLO {"estado": "..."}.

    """


    user_msg = f"""
    Hoy: {today}
    Fecha de recepción: {fecha_recepcion}
    Textos relevantes:
    {textos}
    """

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg}
        ],
        temperature=0.0,
    )

    content = resp.choices[0].message.content.strip()

    # Limpia por si viene con ```json
    if content.startswith("```"):
        content = re.sub(r"^```(?:json)?\s*|\s*```$", "", content)

    try:
        data = json.loads(content)
        return data.get("estado", "desconocido")
    except:
        return "desconocido"


In [79]:
query2="Somos una pyme española del sector agroalimentario y buscamos subvenciones para modernizar nuestras instalaciones y mejorar la eficiencia energética de nuestros procesos de producción, especialmente aquellas que financien la incorporación de energías renovables, digitalización o reducción de emisiones."

In [116]:
data_frame_resumen(query2)

{'descripcion': 'pyme sector agroalimentario subvenciones modernizar instalaciones', 'regiones': [1], 'finalidad': 12, 'instrumentos': 1, 'tiposBeneficiario': [3], 'descripcionTipoBusqueda': 2, 'order': 'fechaRecepcion', 'direccion': 'desc', 'vpd': 'GE', 'page': 0, 'pageSize': 25}


,numeroConvocatoria,descripcion,fechaRecepcion,nivel1,nivel2,presupuesto_total,inicio,final,estado
0,845928,Subvenciones a los seguros de acuicultura mari...,2025-07-14,ESTADO,"MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN","1,000,000.00",2025-06-01,2026-05-31,cerrada-no-publica
1,845921,Subvenciones a los seguros de acuicultura cont...,2025-07-14,ESTADO,"MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN","200,000.00",2025-06-01,2026-05-31,cerrada-no-publica
2,832957,Convocatoria para el otorgamiento de subvencio...,2025-05-15,FIGUERES,AYUNTAMIENTO DE FIGUERES,"12,000.00",2025-05-21,2025-06-14,cerrada-no-publica
3,829278,Resolución de 25 de abril de 2025 del Fondo Es...,2025-04-28,ESTADO,"MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN","33,333,333.30",a partir del día siguiente al de la publicació...,2025-05-29,cerrada
4,821006,"Resolución de 7 de mayo de 2025, de la Secreta...",2025-03-18,ESTADO,"MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN","1,795,000.00",2025-05-14,2025-06-03,cerrada-no-publica
5,819034,Subvenciones a los seguros de acuicultura mari...,2025-03-07,ESTADO,"MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN","50,000.00",2025-02-01,2025-04-30,cerrada-no-publica
6,817590,Orden de 27 de febrero de 2025 del Ministro de...,2025-02-28,ESTADO,"MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN","100,000.00",Desde el día siguiente a la publicación de la ...,10 días hábiles desde el día siguiente a la pu...,cerrada
7,806457,"Subvenciones a los seguros agrícolas, ganadero...",2024-12-30,ESTADO,"MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN","315,000,000.00",2025-01-01,2026-07-31,cerrada-no-publica
8,805589,"Orden de 26 de diciembre de 2024, del MAPA, po...",2024-12-26,ESTADO,"MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN","50,000.00",2025-01-16,2026-09-01,cerrada-no-publica
9,805588,"Orden de 26 de diciembre de 2024, del MAPA, po...",2024-12-26,ESTADO,"MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN","50,000.00",2025-01-16,2026-09-01,cerrada-no-publica


In [93]:
dic=obtener_convocatoria_por_id(793135)
dic

{'id': 994695,
 'organo': {'nivel1': 'ESTADO',
  'nivel2': 'MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN',
  'nivel3': 'SECRETARÍA GENERAL DE PESCA'},
 'sedeElectronica': 'https://sede.mapa.gob.es/portal/site/seMAPA/ficha-procedimiento?procedure_suborg_responsable=27&procedure_etiqueta_pdu=null&procedure_id=475&by=theme',
 'codigoBDNS': '793135',
 'fechaRecepcion': '2024-10-24',
 'instrumentos': [{'descripcion': 'SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPRESTACIÓN '}],
 'tipoConvocatoria': 'Concurrencia competitiva - canónica',
 'presupuestoTotal': 300000,
 'mrr': False,
 'descripcion': 'Orden por la que se convoca para el año 2024 la concesión de ayudas a las organizaciones profesionales, del sector de los productos de la pesca y de la acuicultura para su consolidación',
 'descripcionLeng': None,
 'tiposBeneficiarios': [{'descripcion': 'PYME Y PERSONAS FÍSICAS QUE DESARROLLAN ACTIVIDAD ECONÓMICA'}],
 'sectores': [{'descripcion': 'Pesca y acuicultura', 'codigo': '03'}],
 'regiones':

In [105]:
query3="Subvenciones en el ámbito del comercio, de los servicios, la artesanía y la moda para el año 2025"

In [106]:
data_frame_resumen(query3)

{'descripcion': 'subvenciones comercio servicios artesanía moda', 'finalidad': 14, 'instrumentos': 1, 'descripcionTipoBusqueda': 2, 'order': 'fechaRecepcion', 'direccion': 'desc', 'vpd': 'GE', 'page': 0, 'pageSize': 25}


,numeroConvocatoria,descripcion,fechaRecepcion,nivel1,nivel2,presupuesto_total,inicio,final,estado
0,851062,ACUERDO DE LA JUNTA DE GOBIERNO LOCAL DE 30/07...,2025-08-08,LORCA,AYUNTAMIENTO DE LORCA,"1,200.00",2025-01-01,2025-07-30,cerrada-no-publica
1,851029,ACUERDO DE LA JUNTA DE GOBIERNO LOCAL DE 30/07...,2025-08-08,LORCA,AYUNTAMIENTO DE LORCA,"30,000.00",2025-01-01,2025-07-30,cerrada-no-publica
2,850909,CONVOCATORIA CONCESION DE SUBVENCIONES PLAN IN...,2025-08-08,CABRA,AYUNTAMIENTO DE CABRA,"200,000.00",DESDE EL DIA SIGUIENTE DE LA PUBLICACIÓN EN EL...,FINALIZARÁ EL 30/09/2025,abierta
3,850906,Subvenciones a entes locales para acondicionam...,2025-08-08,COMUNIDAD FORAL DE NAVARRA,"DEPARTAMENTO DE CULTURA, DEPORTE Y TURISMO","600,000.00",El día siguiente al de la publicación de la co...,El 31 de octubre de 2025,cerrada-no-publica
4,850863,SUBV. EXCEPCIONAL FEDERACIÓN GALLEGA DE COMERC...,2025-08-07,DIPUTACIÓN PROV. DE LUGO,DIPUTACIÓN PROVINCIAL DE LUGO,"50,000.00",2025-01-01,2025-12-31,cerrada-no-publica
...,...,...,...,...,...,...,...,...,...
70,847222,SUBV. NOMINAT. FUNCIONES PÚBLICAS ADMINISTRATI...,2025-07-19,ANDALUCÍA,"CONSEJERÍA DE EMPLEO, EMPRESA Y TRABAJO AUTÓNOMO","186,400.00",2025-01-01,2025-12-31,cerrada-no-publica
71,847115,Bases reguladoras de la campaña de fomento del...,2025-07-18,"POBLA, SA","AYUNTAMIENTO DE POBLA, SA","3,500.00",2025-07-01,2025-08-31,abierta
72,847035,CONVOCATORIA ORDINARIA DE SUBVENCIONES PARA LA...,2025-07-18,CIUDAD AUTÓNOMA DE MELILLA,"CONSEJERÍA DE ECONOMÍA, COMERCIO, INNOVACIÓN T...","5,000,000.00",2025-07-19,2025-12-31,abierta
73,846996,Decreto de Presidencia nº 3.749 de 7 de julio ...,2025-07-18,DIPUTACIÓN PROV. DE VALLADOLID,DIPUTACIÓN PROVINCIAL DE VALLADOLID,"215,000.00",2025-01-01,2025-12-31,cerrada-no-publica


In [107]:
obtener_convocatoria_por_id(851062)

{'id': 1052623,
 'organo': {'nivel1': 'LORCA',
  'nivel2': 'AYUNTAMIENTO DE LORCA',
  'nivel3': None},
 'sedeElectronica': None,
 'codigoBDNS': '851062',
 'fechaRecepcion': '2025-08-08',
 'instrumentos': [{'descripcion': 'SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPRESTACIÓN '}],
 'tipoConvocatoria': 'Concesión directa - instrumental',
 'presupuestoTotal': 1200,
 'mrr': False,
 'descripcion': 'ACUERDO DE LA JUNTA DE GOBIERNO LOCAL DE 30/07/2025 SOBRE CONCESIÓN DE SUBVENCIÓN DIRECTA A LA CAMARA DE COMERCIO E INDUSTRIA DE LORCA PARA LA VI JORNADA DE LODERAZGO EMPRESARIAL',
 'descripcionLeng': None,
 'tiposBeneficiarios': [{'descripcion': 'PERSONAS JURÍDICAS QUE NO DESARROLLAN ACTIVIDAD ECONÓMICA'}],
 'sectores': [{'descripcion': 'OTROS SERVICIOS', 'codigo': 'S'}],
 'regiones': [{'descripcion': 'ES62 - REGION DE MURCIA'}],
 'descripcionFinalidad': 'Comercio, Turismo y Pymes',
 'descripcionBasesReguladoras': 'Ordenanza General de las Bases Reguladoras de la Concesión de Subvenciones',
 'urlB

In [109]:
query4="Somos una pequeña empresa de tecnología ubicada en Sevilla que desarrolla soluciones de software para el sector educativo. Queremos encontrar convocatorias de subvenciones actualmente abiertas en Andalucía que financien proyectos de innovación tecnológica, digitalización de procesos o modernización de equipos. Nos interesan ayudas tanto para adquisición de hardware y software como para formación del personal."

In [115]:
data_frame_resumen(query4)

{'descripcion': 'pequeña empresa tecnología Sevilla subvenciones innovación', 'regiones': [71, 63], 'finalidad': 17, 'instrumentos': 1, 'tiposBeneficiario': [3], 'descripcionTipoBusqueda': 2, 'order': 'fechaRecepcion', 'direccion': 'desc', 'vpd': 'GE', 'page': 0, 'pageSize': 25}


,numeroConvocatoria,descripcion,fechaRecepcion,nivel1,nivel2,presupuesto_total,inicio,final,estado
0,842307,"RESOLUCIÓN, DE 7 DE JULIO DE 2025, DE LA D.G D...",2025-06-25,ANDALUCÍA,"CONSEJERÍA DE UNIVERSIDAD, INVESTIGACIÓN E INN...","4,000,000.00",2025-07-12,2025-09-15,abierta
1,824198,Convocatoria Pyme Innova 2025 - Cámara de Sevilla,2025-04-01,OTROS,CÁMARA DE COMERCIO DE SEVILLA,"636,650.00","A las 09:00h, transcurridos 5 días hábiles des...",A las 14:00h,cerrada
2,824158,Convocatoria Pyme Sostenible 2025 - Cámara de ...,2025-04-01,OTROS,CÁMARA DE COMERCIO DE SEVILLA,"149,800.00",2025-04-04,2025-04-10,cerrada
3,822939,Convocatoria Programa Pyme Digital 2025-Cámara...,2025-03-26,OTROS,CÁMARA DE COMERCIO DE SEVILLA,"674,100.00",2025-03-31,2025-04-04,cerrada
4,822594,Convocatoria Programa Pyme Cibersegura 2025. C...,2025-03-25,OTROS,CÁMARA DE COMERCIO DE SEVILLA,"278,200.00",2025-04-04,2025-04-21,cerrada
5,803854,Resolución Dirección de Fundación Biodiversida...,2024-12-18,ESTADO,MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL R...,"11,500,000.00",Al día siguiente de la publicación del extract...,2025-02-20,cerrada


In [111]:
obtener_convocatoria_por_id(824198)

{'id': 1025759,
 'organo': {'nivel1': 'OTROS',
  'nivel2': 'CÁMARA DE COMERCIO DE SEVILLA',
  'nivel3': None},
 'sedeElectronica': 'https://sede.camara.es/sede/sevilla',
 'codigoBDNS': '824198',
 'fechaRecepcion': '2025-04-01',
 'instrumentos': [{'descripcion': 'SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPRESTACIÓN '}],
 'tipoConvocatoria': 'Concurrencia competitiva - canónica',
 'presupuestoTotal': 636650,
 'mrr': False,
 'descripcion': 'Convocatoria Pyme Innova 2025 - Cámara de Sevilla',
 'descripcionLeng': None,
 'tiposBeneficiarios': [{'descripcion': 'PYME Y PERSONAS FÍSICAS QUE DESARROLLAN ACTIVIDAD ECONÓMICA'}],
 'sectores': [{'descripcion': 'ACTIVIDADES ADMINISTRATIVAS Y SERVICIOS AUXILIARES',
   'codigo': 'N'},
  {'descripcion': 'ACTIVIDADES ARTÍSTICAS, RECREATIVAS Y DE ENTRETENIMIENTO',
   'codigo': 'R'},
  {'descripcion': 'ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES DE PERSONAL DOMÉSTICO; ACTIVIDADES DE LOS HOGARES COMO PRODUCTORES DE BIENES Y SERVICIOS PARA USO PROPIO',
   'co